In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Step 1: Load and Preprocess the Data
file_path = 'CarPrice_Assignment.csv'
car_data = pd.read_csv(file_path)
car_data

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [4]:
# Dropping irrelevant columns
car_data = car_data.drop(columns=['car_ID', 'CarName'])
car_data

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,-1,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,-1,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [5]:
# Encoding categorical variables
categorical_columns = car_data.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
for col in categorical_columns:
    car_data[col] = label_encoder.fit_transform(car_data[col])

# Split the data into features (X) and target (y)
X = car_data.drop(columns=['price'])
y = car_data['price']

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test 

(array([[-0.67147172, -3.04138127, -0.46929532, ..., -1.9442653 ,
          0.88574028,  1.20076115],
        [-0.67147172,  0.32879797, -0.46929532, ..., -0.26296022,
         -0.95301169, -0.98276371],
        [-0.67147172,  0.32879797, -0.46929532, ..., -0.68328649,
          0.88574028,  0.9096245 ],
        ...,
        [ 0.133509  ,  0.32879797, -0.46929532, ...,  0.15736606,
          0.88574028,  0.9096245 ],
        [ 1.74347043,  0.32879797, -0.46929532, ...,  0.15736606,
         -0.95301169, -0.98276371],
        [-0.67147172,  0.32879797, -0.46929532, ...,  0.15736606,
         -1.25947035, -1.27390036]]),
 array([[-6.71471721e-01,  3.28797975e-01, -4.69295318e-01,
         -8.84651737e-01,  4.49677322e-01,  1.21333041e+00,
         -1.21866670e-01,  7.89638586e-01,  1.21479941e+00,
          4.63650109e-01,  8.10288200e-01,  1.29846525e+00,
         -1.39082791e-02,  1.11220997e+00,  1.97618448e+00,
          8.69567500e-01,  1.07425208e+00,  4.30217198e-01,
         -5.4

In [7]:
# Step 2: Model Implementation
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
    "Support Vector Regressor": SVR()
}

# Dictionary to store the results
results = {}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        "R-squared": r2,
        "MSE": mse,
        "MAE": mae
    }

# Display the results
for model_name, metrics in results.items():
    print(f"{model_name}:\n R-squared: {metrics['R-squared']}, MSE: {metrics['MSE']}, MAE: {metrics['MAE']}\n")


# Step 4: Feature Importance Analysis (for tree-based models)
importances = models["Random Forest Regressor"].feature_importances_
feature_names = X.columns
feature_importances = pd.DataFrame(importances, index=feature_names, columns=['Importance']).sort_values(by='Importance', ascending=False)
print("Feature Importances:\n", feature_importances)

# Step 5: Hyperparameter Tuning for Random Forest (as an example)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=models["Random Forest Regressor"], param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and performance
print(f"Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Performance of the best model
mse_best = mean_squared_error(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"Best Model Performance:\n R-squared: {r2_best}, MSE: {mse_best}, MAE: {mae_best}")


Linear Regression:
 R-squared: 0.7983838478445056, MSE: 15916389.72543959, MAE: 2526.407450143427

Decision Tree Regressor:
 R-squared: 0.9002445186663295, MSE: 7875098.801265586, MAE: 1939.138219512195

Random Forest Regressor:
 R-squared: 0.957858926292672, MSE: 3326785.802642497, MAE: 1292.9702357723577

Gradient Boosting Regressor:
 R-squared: 0.9275422048207606, MSE: 5720109.695523835, MAE: 1667.525102198733

Support Vector Regressor:
 R-squared: -0.09995143262001083, MSE: 86834588.8633635, MAE: 5695.576436854142

Feature Importances:
                   Importance
enginesize          0.557068
curbweight          0.296978
highwaympg          0.045955
horsepower          0.026930
carwidth            0.014485
carlength           0.009149
wheelbase           0.007700
peakrpm             0.006920
citympg             0.006640
stroke              0.005352
boreratio           0.004605
compressionratio    0.003700
carheight           0.003509
fuelsystem          0.002330
carbody           